In [20]:
from google.cloud import aiplatform
from google import genai
from google.genai import types
from google.adk.agents import LlmAgent
from vertexai.preview.reasoning_engines import AdkApp
import json

In [2]:
prompt = """
Location: //compute.googleapis.com/projects/saifguard-gf-rrag-0/global/backendServices/gf-rrag-0-external-default
Description: The external HTTP(S) Load Balancer's backend service (gf-rrag-0-external-default) does not have a Cloud Armor security policy attached. This directly violates the SAIF recommendation that "Each GCP backend service MUST HAVE a security policy defined." Without a WAF, the application is highly vulnerable to DDoS attacks, SQL Injection, Cross-Site Scripting (XSS), and other OWASP Top 10 web application attacks.
Remediation: Attach a Cloud Armor security policy to the gf-rrag-0-external-default backend service. Configure the policy with pre-configured rules for OWASP Top 10 vulnerabilities and adaptive protection for DDoS mitigation.
"""

In [7]:
gcp_project_tool_response = """
Here\'s a detailed security audit report for project `saifguard-gf-rrag-0` based on the SAIF framework, ordered by severity:\n\n### 🔴 Critical\n\n*   **Vulnerability:** Lack of Web Application Firewall (WAF) / DDoS Protection on External Load Balancer\n    *   **Location:** `//compute.googleapis.com/projects/saifguard-gf-rrag-0/global/backendServices/gf-rrag-0-external-default`\n    *   **Description:** The external HTTP(S) Load Balancer\'s backend service (`gf-rrag-0-external-default`) does not have a Cloud Armor security policy attached. This directly violates the SAIF recommendation that "Each GCP backend service MUST HAVE a security policy defined." Without a WAF, the application is highly vulnerable to DDoS attacks, SQL Injection, Cross-Site Scripting (XSS), and other OWASP Top 10 web application attacks.\n    *   **Remediation:** Attach a Cloud Armor security policy to the `gf-rrag-0-external-default` backend service. Configure the policy with pre-configured rules for OWASP Top 10 vulnerabilities and adaptive protection for DDoS mitigation.\n\n*   **Vulnerability:** Disabled Backups for Cloud SQL Instance\n    *   **Location:** `//cloudsql.googleapis.com/projects/saifguard-gf-rrag-0/instances/gf-rrag-0`\n    *   **Description:** The Cloud SQL instance `gf-rrag-0` has automated backups disabled (`settings.backupConfiguration.enabled: false`). This poses a critical risk to data availability and integrity. In the event of data corruption, accidental deletion, or a ransomware attack, data recovery would be severely hampered or impossible, leading to significant downtime and potential data loss. This violates SAIF\'s "Availability Risks" and "Integrity Risks" categories.\n    *   **Remediation:** Enable automated backups for the `gf-rrag-0` Cloud SQL instance. Configure a suitable backup schedule and retention policy to meet the organization\'s Recovery Point Objective (RPO) and Recovery Time Objective (RTO).\n\n*   **Vulnerability:** Cloud SQL Not Requiring SSL for Connections\n    *   **Location:** `//cloudsql.googleapis.com/projects/saifguard-gf-rrag-0/instances/gf-rrag-0`\n    *   **Description:** The Cloud SQL instance `gf-rrag-0` does not enforce SSL for database connections (`ipConfiguration.requireSsl: false`). While Private Service Connect (PSC) is used for private connectivity, not requiring SSL means that data in transit between the Cloud SQL Proxy and the database could potentially be intercepted or tampered with if the internal network is compromised. This is a Sensitive Data Exposure (A03) vulnerability.\n    *   **Remediation:** Enable `requireSsl` for the `gf-rrag-0` Cloud SQL instance to ensure all connections are encrypted, providing defense in depth for data in transit.\n\n*   **Vulnerability:** Disabled Versioning on Critical Storage Buckets\n    *   **Location:**\n        *   `//storage.googleapis.com/tdd_movies_app` (`versioning: {}`)\n        *   `//storage.googleapis.com/saifguard-gf-rrag-0-iac-state` (`versioning: false`)\n    *   **Description:** Versioning is disabled for both the `tdd_movies_app` data bucket and the `saifguard-gf-rrag-0-iac-state` Infrastructure as Code (IaC) state bucket. This is a critical data integrity and recovery risk (A08 - Software and Data Integrity Failures). Without versioning, accidental deletion, malicious modification, or ransomware attacks could lead to irreversible data loss or compromise of the application\'s infrastructure state.\n    *   **Remediation:** Enable object versioning for both `tdd_movies_app` and `saifguard-gf-rrag-0-iac-state` buckets. Implement appropriate lifecycle policies to manage storage costs while retaining necessary historical versions.\n\n### 🟠 High\n\n*   **Vulnerability:** Broad Egress Permissions for Cloud Run Services/Jobs\n    *   **Location:**\n        *   `//run.googleapis.com/projects/saifguard-gf-rrag-0/locations/europe-west1/executions/gf-rrag-0-ingestion-49kfj` (and all other `gf-rrag-0-ingestion` executions)\n        *   `//run.googleapis.com/projects/saifguard-gf-rrag-0/locations/europe-west1/services/gf-rrag-0-frontend`\n    *   **Description:** The Cloud Run services and jobs are configured with `run.googleapis.com/vpc-access-egress: all-traffic`. This allows the containers to initiate connections to any external IP address. If a container is compromised (e.g., via a supply chain attack, injection flaw, or insecure deserialization), this broad egress permission could be exploited for data exfiltration, command and control (C2) communication, or launching further attacks. This is an Insecure Design (A04) and Security Misconfiguration (A05).\n    *   **Remediation:** Restrict egress traffic from Cloud Run services and jobs to only necessary destinations. Utilize VPC Service Controls, Private Google Access, or more granular firewall rules within the VPC network to limit outbound connections.\n\n*   **Vulnerability:** Use of Generic/Placeholder Images in Production-like Cloud Run Deployments\n    *   **Location:**\n        *   `//run.googleapis.com/projects/saifguard-gf-rrag-0/locations/europe-west1/executions/gf-rrag-0-ingestion-49kfj` (and all other `gf-rrag-0-ingestion` executions, `ingestion` container image)\n        *   `//run.googleapis.com/projects/saifguard-gf-rrag-0/locations/europe-west1/services/gf-rrag-0-frontend` (`frontend` container image)\n    *   **Description:** Both the `gf-rrag-0-ingestion` job and the `gf-rrag-0-frontend` service are using the generic `us-docker.pkg.dev/cloudrun/container/hello` image. This "hello world" image is not intended for production applications. Deploying it in a production-like environment (especially the frontend, which is exposed via an external load balancer) is a significant Security Misconfiguration (A05). It suggests either a non-functional deployment or a placeholder that has not been replaced, making it impossible to assess the actual application\'s security posture.\n    *   **Remediation:** Replace the generic `us-docker.pkg.dev/cloudrun/container/hello` images with the actual, production-ready application images. Ensure that all deployed images are hardened, scanned for vulnerabilities, and built from trusted sources.\n\n### 🟡 Medium\n\n*   **Vulnerability:** Repeated Cloud Run Job Timeouts\n    *   **Location:** Multiple `run.googleapis.com/Execution` resources for `gf-rrag-0-ingestion` job (e.g., `gf-rrag-0-ingestion-49kfj`, `gf-rrag-0-ingestion-g7p4d`, etc.)\n    *   **Description:** Numerous executions of the `gf-rrag-0-ingestion` Cloud Run job are consistently failing with "The configured timeout was reached" messages. While primarily a functional issue, repeated timeouts can indicate underlying problems that could have security implications, such as resource exhaustion (SAIF - Resource Exhaustion/Denial of Service) or a process hanging due to unexpected or malicious input. It also leads to wasted compute resources.\n    *   **Remediation:** Investigate the root cause of the persistent timeouts. This may involve reviewing application logs, optimizing the ingestion logic, or adjusting resource allocations (CPU/memory) and `timeoutSeconds` for the job. Ensure the application handles long-running operations gracefully.\n\n*   **Vulnerability:** Inherited Public Access Prevention for IaC State Bucket\n    *   **Location:** `//storage.googleapis.com/saifguard-gf-rrag-0-iac-state` (`publicAccessPrevention: inherited`)\n    *   **Description:** The IaC state bucket `saifguard-gf-rrag-0-iac-state` has `publicAccessPrevention` set to `inherited` instead of `enforced`. While `bucketPolicyOnly` and `uniformBucketLevelAccess` are enabled (which is good), relying on inherited settings for public access prevention for sensitive IaC state is a Security Misconfiguration (A05). It introduces a dependency on project-level settings, which might be inadvertently changed or less strict.\n    *   **Remediation:** Explicitly set `publicAccessPrevention` to `enforced` for the `saifguard-gf-rrag-0-iac-state` bucket to ensure consistent and robust protection against public exposure.\n\n*   **Vulnerability:** Overly Permissive Default Compute Service Account\n    *   **Location:** `//iam.googleapis.com/projects/saifguard-gf-rrag-0/serviceAccounts/74480264432-compute@developer.gserviceaccount.com`\n    *   **Description:** The default Compute Engine service account is present and likely has broad default permissions (e.g., Editor role). While specific custom service accounts are also used, relying on or not restricting the default service account is a common Security Misconfiguration (A05) and can lead to Broken Access Control (A01). If a Compute Engine instance using this default account is compromised, the attacker could gain extensive control over the project.\n    *   **Remediation:** Review the IAM policies for the default Compute Engine service account. Remove any overly permissive roles and grant only the minimum necessary permissions. For new resources, always prefer creating and assigning custom service accounts with fine-grained roles.
"""

In [15]:
PROJECT_ID = "saifguard-gf-rrag-0"
REGION = "europe-west4"
MODEL = "gemini-2.5-flash-lite"


In [4]:
SYSTEM_PROMPT = """
A vulnerability description from a GCP project is given to you. You need to extract the google cloud console URL of the resource to
allow the user to click on it.
"""

In [8]:
SYSTEM_PROMPT = """
<INSTRUCTIONS>
A list of vulnerability descriptions from a GCP project is given to you. 
You need to extract the google cloud console URL for each vulnerable resources to allow the user to click on it.
</INSTRUCTIONS>

<OUTPUT>
Output a List[str] which is the list of resources URL
</OUTPUT>
"""

In [16]:
from typing import List

from pydantic import BaseModel


class Links(BaseModel):
    links: List[str]

In [17]:
query = f"""
# Vulnerabilities
{gcp_project_tool_response}
"""

contents = [
    types.Part.from_text(text=query),
]

client = genai.Client(
    vertexai=True, project=PROJECT_ID, location=REGION,
)
response = client.models.generate_content(
    model=MODEL,
    contents=contents,
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_PROMPT,
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=Links,
    ),
)

In [18]:
response.text

'{\n  "links": [\n    "https://console.cloud.google.com/net-services/loadbalancing/backendServices/details/gf-rrag-0-external-default?project=saifguard-gf-rrag-0",\n    "https://console.cloud.google.com/sql/instances/gf-rrag-0/overview?project=saifguard-gf-rrag-0",\n    "https://console.cloud.google.com/sql/instances/gf-rrag-0/overview?project=saifguard-gf-rrag-0",\n    "https://console.cloud.google.com/storage/browser/tdd_movies_app?project=saifguard-gf-rrag-0&page=versioning",\n    "https://console.cloud.google.com/storage/browser/saifguard-gf-rrag-0-iac-state?project=saifguard-gf-rrag-0&page=versioning",\n    "https://console.cloud.google.com/run/jobs/gf-rrag-0-ingestion/executions?project=saifguard-gf-rrag-0&region=europe-west1",\n    "https://console.cloud.google.com/run/services/gf-rrag-0-frontend?project=saifguard-gf-rrag-0&region=europe-west1",\n    "https://console.cloud.google.com/run/jobs/gf-rrag-0-ingestion/executions?project=saifguard-gf-rrag-0&region=europe-west1",\n    "

In [21]:
urls = json.loads(response.text)

In [22]:
urls

{'links': ['https://console.cloud.google.com/net-services/loadbalancing/backendServices/details/gf-rrag-0-external-default?project=saifguard-gf-rrag-0',
  'https://console.cloud.google.com/sql/instances/gf-rrag-0/overview?project=saifguard-gf-rrag-0',
  'https://console.cloud.google.com/sql/instances/gf-rrag-0/overview?project=saifguard-gf-rrag-0',
  'https://console.cloud.google.com/storage/browser/tdd_movies_app?project=saifguard-gf-rrag-0&page=versioning',
  'https://console.cloud.google.com/storage/browser/saifguard-gf-rrag-0-iac-state?project=saifguard-gf-rrag-0&page=versioning',
  'https://console.cloud.google.com/run/jobs/gf-rrag-0-ingestion/executions?project=saifguard-gf-rrag-0&region=europe-west1',
  'https://console.cloud.google.com/run/services/gf-rrag-0-frontend?project=saifguard-gf-rrag-0&region=europe-west1',
  'https://console.cloud.google.com/run/jobs/gf-rrag-0-ingestion/executions?project=saifguard-gf-rrag-0&region=europe-west1',
  'https://console.cloud.google.com/ru